# Data import and cleansing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
ca2 = pd.read_csv('banking.csv', header=0)
ca2 = ca2.dropna()
print(ca2.shape)
print(list(ca2.columns))

(41188, 21)
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']


In [3]:
cadrop=ca2[['contact', 'month', 'day_of_week', 'duration', 'campaign']]
cadrop

,contact,month,day_of_week,duration,campaign
0,cellular,aug,thu,210,1
1,cellular,nov,fri,138,1
2,cellular,jun,thu,339,3
3,cellular,apr,fri,185,2
4,cellular,aug,fri,137,1
...,...,...,...,...,...
41183,telephone,jun,thu,222,1
41184,telephone,may,thu,196,2
41185,telephone,may,wed,62,3
41186,telephone,oct,tue,200,2


In [4]:
#import seaborn as sns
#sns.pairplot(ca2, hue='y')

In [5]:
ca= ca2.drop(['contact', 'month', 'day_of_week', 'duration', 'campaign'],axis=1)
#ca= ca2.drop([ 'month', 'day_of_week', 'duration'],axis=1)

ca
#'contact', 'month', 'day_of_week', 'duration', 'campaign','nr_employed''cons_conf_idx','emp_var_rate'

,age,job,marital,education,default,housing,loan,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,59,retired,married,high.school,unknown,no,yes,999,0,nonexistent,1.4,94.465,-41.8,4.866,5228.1,0
41184,31,housemaid,married,basic.4y,unknown,no,no,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,0
41185,42,admin.,single,university.degree,unknown,yes,yes,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
41186,48,technician,married,professional.course,no,no,yes,999,0,nonexistent,-3.4,92.431,-26.9,0.742,5017.5,0


In [6]:
ca.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                   int64
dtype: object

In [7]:
base = ca[['y']]

In [8]:
## identify categorical variables
cat = ca.select_dtypes(exclude=['int', 'float']).columns
print(len(cat))
print('Number of categoriacal features:', len(cat))

CatVar=ca[cat]
Nl=[]
for column in CatVar.columns:
    if CatVar[column].isnull().values.any() == True:
        Nl.append(column)
    
print('columns with missing values:', Nl)

7
Number of categoriacal features: 7
columns with missing values: []


In [9]:
for column in CatVar.columns:
    print(CatVar.groupby(column)[column].agg(['count']))

               count
job                 
admin.         10422
blue-collar     9254
entrepreneur    1456
housemaid       1060
management      2924
retired         1720
self-employed   1421
services        3969
student          875
technician      6743
unemployed      1014
unknown          330
          count
marital        
divorced   4612
married   24928
single    11568
unknown      80
                     count
education                 
basic.4y              4176
basic.6y              2292
basic.9y              6045
high.school           9515
illiterate              18
professional.course   5243
university.degree    12168
unknown               1731
         count
default       
no       32588
unknown   8597
yes          3
         count
housing       
no       18622
unknown    990
yes      21576
         count
loan          
no       33950
unknown    990
yes       6248
             count
poutcome          
failure       4252
nonexistent  35563
success       1373


In [10]:
CatVar['education']=np.where(CatVar['education']=='basic.4y', 'basic',
                            np.where(CatVar['education']=='basic.6y', 'basic',
                                    np.where(CatVar['education']=='basic.9y', 'basic',
                                            np.where(CatVar['education']=='illiterate', 'unknown',
                                                    CatVar['education']))))

CatVar['job']=np.where(CatVar['job']=='entrepreneur', 'management',
                      np.where(CatVar['job']=='self-employed', 'management',
                              np.where(CatVar['job']=='retired', 'other 1', 
                                       np.where(CatVar['job']=='unknown', 'other 2', 
                                                np.where(CatVar['job']=='student', 'other 1', 
                                                         np.where(CatVar['job']=='unemployed', 'other 2', 
                                                                  np.where(CatVar['job']=='housemaid', 'other 2', 
                                                                           CatVar['job'])))))))


CatVar['marital']=np.where(CatVar['marital']=='unknown','single',CatVar['marital'])


CatVar['default']=np.where(CatVar['default']=='yes','unknown',CatVar['default'])

/Users/szymanskia1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/szymanskia1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/szymanskia1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [11]:
for column in CatVar.columns:
    print(CatVar.groupby(column)[column].agg(['count']))

             count
job               
admin.       10422
blue-collar   9254
management    5801
other 1       2595
other 2       2404
services      3969
technician    6743
          count
marital        
divorced   4612
married   24928
single    11648
                     count
education                 
basic                12513
high.school           9515
professional.course   5243
university.degree    12168
unknown               1749
         count
default       
no       32588
unknown   8600
         count
housing       
no       18622
unknown    990
yes      21576
         count
loan          
no       33950
unknown    990
yes       6248
             count
poutcome          
failure       4252
nonexistent  35563
success       1373


In [12]:
Cat_dummies = pd.get_dummies(CatVar, drop_first=True)
print(CatVar.shape)
print(Cat_dummies.shape)

(41188, 7)
(41188, 19)


In [13]:
## Identify continuous variables
cont1 = ca.select_dtypes(include=['int', 'float']).columns
print('initial number of continuous features: ',len(cont1))


## Exclude those previously classified as dummies
cont4=set(cont1).difference(cat)
cont=set(cont4).difference(base)
print('initial number of continuous features after exclusions: ',len(cont))
print()

ContVar= ca[cont]

Nl3=[]
for column in ContVar.columns:
    if ContVar[column].isnull().values.any() == True:
        Nl3.append(column)
    
print('features with missing values: ',Nl3)

initial number of continuous features:  9
initial number of continuous features after exclusions:  8

features with missing values:  []


### Standardise continuous variables

In [14]:
from sklearn.preprocessing import StandardScaler

### Standardise variables 
sc=StandardScaler()

sc.fit(ContVar)
ContVar_std=pd.DataFrame(sc.transform(ContVar),columns=cont)

### double check for missing values
Nl4=[]
for column in ContVar_std.columns:
    if ContVar_std[column].isnull().values.any() == True:
        Nl4.append(column)
    
print('Variables with missing in ContVar_std: ',Nl4)

Variables with missing in ContVar_std:  []


In [15]:
print(base.shape)
print(ContVar.shape)
print(ContVar_std.shape)
print(Cat_dummies.shape)

(41188, 1)
(41188, 8)
(41188, 8)
(41188, 19)


In [16]:
#df = pd.concat([base,ContVar,Cat_dummies,],axis=1)
df = pd.concat([base,ContVar_std,Cat_dummies],axis=1)
df.shape
df.to_csv('banking_cleansed.csv')
print(df.shape)

(41188, 28)
